In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np

In [2]:
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [3]:
with open('id_host.pickle', 'rb') as file:
    id_url_std = pickle.load(file)

In [4]:
url_cl_info = dict()
for key in id_url_std:
    url_cl_info[key] = URL_info()

In [5]:
listOfFiles = os.listdir('rh_host_data_upd')
idx = [int(str2[3:-4]) for str2 in listOfFiles]
idx.sort()
best_dat = []
for idx1 in tqdm(idx):
    with zipfile.ZipFile('rh_host_data_upd/rh_{}.zip'.format(idx1)) as myzip:
        with myzip.open('rh_{}.pickle'.format(idx1), 'r') as file:
            obj = pickle.load(file)
            for session in obj:
                #чтсло первых кликов
                if len(session[3])>0:
                    if session[3][0]>-1:
                        url_cl_info[session[3][0]].num_first_clicks+=1
                #число последних кликов  
                if len(session[3])>0:
                    if session[3][-1]>-1:
                        url_cl_info[session[3][-1]].num_last_clicks+=1
                #число показов
                show_ps = dict()
                for i in range(len(session[2])):
                    show_ps[session[2][i]]=i
                    if session[2][i]>-1:
                        url_cl_info[session[2][i]].total_shows+=1
                        #показы на первых 10-ти позициях
                        if i<10:
                            url_cl_info[session[2][i]].show_poses[i]+=1
                #число кликов    
                for i in range(len(session[3])):
                    if session[3][i]>-1:
                        url_cl_info[session[3][i]].total_clicks+=1
                        #число кликов после/до клика на этот
                        url_cl_info[session[3][i]].clicks_before_this += i
                        url_cl_info[session[3][i]].clicks_after_this += (len(session[3])-1-i)
                        #клики на первых 10-ти позициях
                        if show_ps[session[3][i]]<10:
                            url_cl_info[session[3][i]].click_poses[show_ps[session[3][i]]]+=1
                #время
                if len(session[4])>1:
                    for i in range(len(session[4])-2):
                        if session[3][i]>-1:
                            url_cl_info[session[3][i]].time += ((session[4][i+1]-session[4][i])/1000)
                pos_list = []
                for key in show_ps:
                    pos_list.append(show_ps[key])
                pos_list = np.array(pos_list)
                clicked_set = set()
                for i in range(len(session[3])):
                    if session[3][i]>-1:
                        cur_pos = show_ps[session[3][i]]
                        clicked_set.add(session[3][i])
                        a1 = np.sum(pos_list<cur_pos)
                        url_cl_info[session[3][i]].clicks_on_lower_pos+=a1
                        if a1>0:
                            url_cl_info[session[3][i]].is_click_on_lower_pos+=1
                        b1 = np.sum(pos_list>cur_pos)
                        url_cl_info[session[3][i]].clicks_on_higher_pos+=b1
                        if b1>0:
                            url_cl_info[session[3][i]].is_click_on_higher_pos+=1
                for i in range(len(session[3])):
                    if session[3][i] in clicked_set and i<10:
                        url_cl_info[session[3][i]].show_poses_cl[i]+=1     

100%|██████████| 1900/1900 [14:58:35<00:00, 28.38s/it]    


In [6]:
with open("host_simple.pickle", 'wb') as file:
    pickle.dump(url_cl_info, file)

In [7]:
1+2

3